In [13]:
# NSMC 한국어 텍스트 전처리 파이프라인
# Day 4: 전처리 + TOP 3 모델 비교 (모듈화 적용 버전)

print("✅ 라이브러리 임포트 완료!")
print(f"scikit-learn 버전: {__import__('sklearn').__version__}")

# =============================================================================
# 0. 환경 설정 (모듈화된 함수 사용)
# =============================================================================

from pathlib import Path
import sys
sys.path.append(r'D:\dataAnalystBook\my_workspace\movie_review') # project_root 경로
from src.utils.setup import setup_notebook_environment

"""
setup_notebook_environment() 함수가 수행하는 작업:
1. 프로젝트 루트 자동 탐지 및 sys.path 추가
2. 한글 폰트 설정 (matplotlib에서 한글 표시)
3. pandas 표시 옵션 설정 (최대 행/열 수 조정)
4. seaborn 스타일 및 색상 팔레트 설정
5. 경고 메시지 숨김 설정
6. 주요 패키지 버전 확인 및 출력
→ 결과: 모든 노트북에서 일관된 환경 구성
"""
setup_notebook_environment()

# =============================================================================
# 1. 데이터 로딩 (모듈화된 함수 사용)
# =============================================================================

sys.path.append(str(project_root))
from src.data.data_loader import load_nsmc_data

"""
load_nsmc_data() 함수가 수행하는 작업:
1. 데이터 디렉토리 자동 탐지 (data/raw/ 폴더 찾기)
2. 다중 인코딩 시도 (utf-8, cp949, euc-kr)
3. 데이터 검증 (컬럼명, 데이터 크기, 라벨값 확인)
4. 결측값 및 데이터 품질 체크
5. 상세한 데이터 요약 정보 출력
→ 결과: 안전하고 검증된 NSMC 데이터 로딩 완료
"""
train_df, test_df = load_nsmc_data()

print(train_df.head())

# 기본 정보 확인
print("\n📋 데이터 기본 정보:")
print(f"훈련 데이터: {train_df.shape}")
print(f"테스트 데이터: {test_df.shape}")
print(f"컬럼: {train_df.columns.tolist()}")

# =============================================================================
# 2. 한국어 텍스트 전처리 (모듈화된 함수 사용)
# =============================================================================

from src.data.text_preprocessor import preprocess_text_data

"""
preprocess_text_data() 함수가 수행하는 작업:
1. 텍스트 정제 (clean_korean_text 함수 내부 호출):
   - 반복 문자 정규화 (ㅋㅋㅋ → ㅋㅋ, !!! → !!)
   - 의미없는 특수문자 제거 (감정 표현은 보존)
   - 여러 공백을 하나로 통합
   - 앞뒤 공백 제거
2. 빈 텍스트 제거 및 개수 리포트
3. 텍스트 길이 계산 및 통계 출력
4. 진행 상황 표시 (tqdm 프로그레스 바)
→ 결과: 'cleaned_text', 'text_length' 컬럼이 추가된 정제된 데이터프레임
"""

print("\n" + "="*50)
print("🔧 데이터 전처리 시작")
print("="*50)

# 훈련 데이터 전처리
print("\n📊 훈련 데이터 전처리...")
train_processed = preprocess_text_data(train_df)

# 테스트 데이터 전처리  
print("\n📊 테스트 데이터 전처리...")
test_processed = preprocess_text_data(test_df)

# 전처리 결과 확인
print("\n✅ 전처리 완료!")
print(f"훈련 데이터: {len(train_processed):,}개")
print(f"테스트 데이터: {len(test_processed):,}개")

# 전처리 예시 확인
print("\n📝 전처리 예시:")
for i in range(3):
    original = train_df.iloc[i]['document']
    cleaned = train_processed.iloc[i]['cleaned_text']
    print(f"\n예시 {i+1}:")
    print(f"원본: {original}")
    print(f"정제: {cleaned}")

# =============================================================================
# 3. 데이터 분할 및 TF-IDF 벡터화
# =============================================================================

print("\n" + "="*50)
print("🔀 데이터 분할 및 벡터화")
print("="*50)

# 텍스트와 라벨 분리
X_train_text = train_processed['cleaned_text'].values
y_train = train_processed['label'].values
X_test_text = test_processed['cleaned_text'].values  
y_test = test_processed['label'].values

from sklearn.model_selection import train_test_split

"""
train_test_split() 함수가 수행하는 작업:
1. 훈련 데이터를 train/validation으로 8:2 분할
2. stratify=y_train으로 클래스 비율 유지
3. random_state=42로 재현 가능한 분할
→ 결과: 균형잡힌 train/validation 데이터셋 생성
"""

# 훈련 데이터를 train/validation으로 분할
X_train, X_val, y_train_split, y_val = train_test_split(
    X_train_text, y_train, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_train
)

print(f"📊 데이터 분할 결과:")
print(f"훈련: {len(X_train):,}개")
print(f"검증: {len(X_val):,}개") 
print(f"테스트: {len(X_test_text):,}개")

# TF-IDF 벡터화
print(f"\n🔢 TF-IDF 벡터화 중...")

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

"""
TfidfVectorizer 설정 및 변환 과정:
1. TF-IDF 설정:
   - max_features=10000: 최대 10,000개 특성으로 제한
   - min_df=2: 최소 2개 문서에 등장하는 단어만 사용
   - max_df=0.95: 95% 이상 문서에 등장하는 흔한 단어 제거
   - ngram_range=(1,2): 단어(1-gram)와 2단어 조합(2-gram) 사용
   - sublinear_tf=True: TF 값에 로그 스케일링 적용

2. 변환 과정:
   - fit_transform(X_train): 훈련 데이터로 어휘 학습 + 벡터 변환
   - transform(X_val, X_test): 학습된 어휘로 검증/테스트 데이터 변환
   
→ 결과: 희소 행렬 형태의 TF-IDF 벡터 (문서 x 단어 특성)
"""

# TF-IDF 설정 (한국어 특화)
from src.features.tfidf_vectorizer import create_tfidf_vectorizer, show_tfidf_stats

# TF-IDF 벡터라이저 생성
tfidf = create_tfidf_vectorizer()

# 훈련 데이터로 TF-IDF 학습 및 변환
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test_text)

print(f"✅ TF-IDF 벡터화 완료!")
print(f"🔢 특성 수: {X_train_tfidf.shape[1]:,}개")
print(f"📊 희소성: {(1 - X_train_tfidf.nnz / np.prod(X_train_tfidf.shape)) * 100:.1f}%")

# 주요 특성 확인
feature_names = tfidf.get_feature_names_out()
print(f"\n📝 주요 특성 예시 (처음 20개):")
print(feature_names[:20])

print(f"\n🎯 전처리 단계 완료! 이제 모델 학습을 시작합니다.")
print(f"⏰ 소요 시간: 약 5-10분 예상")

# =============================================================================
# 4. 모듈화 효과 요약
# =============================================================================

print(f"\n📋 모듈화된 함수들이 수행한 작업 요약:")
print(f"✅ setup_notebook_environment(): 환경 설정 자동화")
print(f"   → 프로젝트 경로, 폰트, pandas 옵션, 패키지 버전 확인")
print(f"✅ load_nsmc_data(): 안전한 데이터 로딩")
print(f"   → 자동 경로 탐지, 인코딩 처리, 데이터 검증")
print(f"✅ preprocess_text_data(): 한국어 특화 전처리")
print(f"   → 텍스트 정제, 빈 값 제거, 길이 계산, 통계 출력")

print(f"\n🎯 최종 준비 완료:")
print(f"  - 정제된 텍스트 데이터: {len(train_processed):,} + {len(test_processed):,}개")
print(f"  - TF-IDF 벡터: {X_train_tfidf.shape} (훈련)")
print(f"  - 검증/테스트 벡터: {X_val_tfidf.shape}, {X_test_tfidf.shape}")
print(f"  - 다음 단계: 머신러닝 모델 학습 (나이브베이즈, 로지스틱회귀, SVM)")

INFO:src.data.data_loader:✅ 데이터 폴더 발견: D:\dataAnalystBook\my_workspace\movie_review\notebooks\02_data_preprocessing\..\..\data\raw
INFO:src.data.data_loader:📊 ratings_train.txt 로딩 시도 (인코딩: utf-8)...


✅ 라이브러리 임포트 완료!
scikit-learn 버전: 1.6.1
🔧 노트북 환경 설정 시작...
✅ 프로젝트 루트 설정: D:\dataAnalystBook\my_workspace\movie_review
✅ 한글 폰트 설정 완료: Malgun Gothic
✅ seaborn 스타일 설정 완료: whitegrid, husl
✅ pandas 옵션 설정 완료: 최대 100행, 전체컬럼 표시
✅ 경고 메시지 숨김 설정 완료

📋 설치된 패키지 버전:
📦 pandas: 2.3.2
📦 numpy: 2.0.1
📦 matplotlib: 3.9.2
📦 seaborn: 0.13.2
📦 scikit-learn: 1.6.1
📦 tqdm: 4.67.1
✅ 환경 설정 완료! (5/5 성공)
📁 데이터 파일 확인:
   훈련 데이터: ..\..\data\raw\ratings_train.txt (14,628,807 bytes)
   테스트 데이터: ..\..\data\raw\ratings_test.txt (4,893,335 bytes)


INFO:src.data.data_loader:✅ ratings_train.txt 로딩 성공! (인코딩: utf-8)
INFO:src.data.data_loader:📊 ratings_test.txt 로딩 시도 (인코딩: utf-8)...
INFO:src.data.data_loader:✅ ratings_test.txt 로딩 성공! (인코딩: utf-8)
INFO:src.data.data_loader:🔍 train 데이터 검증 중...
document    5
dtype: int64
INFO:src.data.data_loader:✅ train 데이터 검증 완료
INFO:src.data.data_loader:🔍 test 데이터 검증 중...
document    3
dtype: int64
INFO:src.data.data_loader:✅ test 데이터 검증 완료


📊 NSMC 데이터셋 요약
   전체 리뷰 수: 200,000개
   훈련 데이터: 150,000개 (75.0%)
   테스트 데이터: 50,000개 (25.0%)

📈 감정 분포:
   훈련 - 긍정: 74,827개 (49.9%), 부정: 75,173개 (50.1%)
   테스트 - 긍정: 25,173개 (50.3%), 부정: 24,827개 (49.7%)

👀 훈련 데이터 미리보기:
         id                           document  label
0   9976970                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                  너무재밓었다그래서보는것을추천한다      0
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1

📋 데이터 기본 정보:
훈련 데이터: (150000, 3)
테스트 데이터: (50000, 3)
컬럼: ['id', 'document', 'label']

🔧 데이터 전처리 시작

📊 훈련 데이터 전처리...
📝 150000개 텍스트 전처리 시작...


텍스트 정제: 100%|███████████████████████████████████████████████████████| 150000/150000 [00:01<00:00, 78840.40it/s]


📊 빈 텍스트 제거: 150,000 → 149,954 (46개 제거)
📏 평균 길이: 34.9자
📏 중앙값: 26.5자

📊 테스트 데이터 전처리...
📝 50000개 텍스트 전처리 시작...


텍스트 정제: 100%|█████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 77580.84it/s]


📊 빈 텍스트 제거: 50,000 → 49,980 (20개 제거)
📏 평균 길이: 35.0자
📏 중앙값: 26.0자

✅ 전처리 완료!
훈련 데이터: 149,954개
테스트 데이터: 49,980개

📝 전처리 예시:

예시 1:
원본: 아 더빙.. 진짜 짜증나네요 목소리
정제: 아 더빙.. 진짜 짜증나네요 목소리

예시 2:
원본: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
정제: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나

예시 3:
원본: 너무재밓었다그래서보는것을추천한다
정제: 너무재밓었다그래서보는것을추천한다

🔀 데이터 분할 및 벡터화
📊 데이터 분할 결과:
훈련: 119,963개
검증: 29,991개
테스트: 49,980개

🔢 TF-IDF 벡터화 중...
✅ TF-IDF 벡터화 완료!
🔢 특성 수: 10,000개
📊 희소성: 100.0%

📝 주요 특성 예시 (처음 20개):
['007' '0개는' '0점' '0점도' '0점은' '0점은 없나' '0점을' '0점이' '10' '10 10' '100'
 '100점' '10개' '10년' '10년도' '10년이' '10년이 지난' '10년전' '10년전에' '10대']

🎯 전처리 단계 완료! 이제 모델 학습을 시작합니다.
⏰ 소요 시간: 약 5-10분 예상

📋 모듈화된 함수들이 수행한 작업 요약:
✅ setup_notebook_environment(): 환경 설정 자동화
   → 프로젝트 경로, 폰트, pandas 옵션, 패키지 버전 확인
✅ load_nsmc_data(): 안전한 데이터 로딩
   → 자동 경로 탐지, 인코딩 처리, 데이터 검증
✅ preprocess_text_data(): 한국어 특화 전처리
   → 텍스트 정제, 빈 값 제거, 길이 계산, 통계 출력

🎯 최종 준비 완료:
  - 정제된 텍스트 데이터: 149,954 + 49,980개
  - TF-IDF 벡터: (119963, 10000) (훈련)
  - 검증/테스트 벡터: (29991, 1000